# Confronto: insertion-sort vs counting-sort

Notebook pratico che confronta le prestazioni di insertion-sort e counting-sort su array di interi. Le celle alternano brevi spiegazioni (markdown) e codice eseguibile.

## Import e impostazioni
Impostiamo l'ambiente: seed riproducibile e stile per i grafici.

In [21]:
%matplotlib inline
import time
import numpy as np
import matplotlib.pyplot as plt
RNG = np.random.default_rng(42)
plt.style.use('seaborn-v0_8-darkgrid')

## Insertion sort
Implementazione semplice (opera su una copia). Complessità: O(n^2).

In [22]:
def insertion_sort(a):
    """Ordina `a` e ritorna una nuova lista ordinata.
    Non modifica l'input."""
    if isinstance(a, np.ndarray):
        arr = a.tolist()
    else:
        arr = list(a)
    n = len(arr)
    for i in range(1, n):
        key = arr[i]
        j = i - 1
        while j >= 0 and arr[j] > key:
            arr[j+1] = arr[j]
            j -= 1
        arr[j+1] = key
    return arr

### Teoria: insertion-sort
- Complessità temporale: O(n) nel caso migliore (array già ordinato), O(n^2) nel caso medio e nel caso peggiore.
- Spazio: O(1) addizionale per la versione in-place; la versione del notebook copia l'input (O(n)).
- Stabilità: insertion sort è stabile se implementato in modo che sposti gli elementi senza alterare l'ordine relativo dei pari chiave.
- Quando usarlo: ottimo per array piccoli o quasi ordinati (poche inversioni). Spesso usato come base case in algoritmi ibridi (es. timsort).

Esempio rapido di `insertion_sort`.

In [23]:
sample = RNG.integers(0, 50, size=12)
print('sample:', sample)
print('insertion_sort result:', insertion_sort(sample))

sample: [ 4 38 32 21 21 42  4 34 10  4 26 48]
insertion_sort result: [4, 4, 4, 10, 21, 21, 26, 32, 34, 38, 42, 48]


## Counting sort
Counting sort per interi non negativi. Complessità: O(n + k).

In [25]:
def counting_sort(a):
    """Counting sort classico che calcola internamente il valore massimo.
    - Accetta liste o numpy array.
    - Supporta solo interi non negativi.
    - Solleva ValueError per valori negativi e MemoryError se k (max+1) è troppo grande.
    """
    if isinstance(a, np.ndarray):
        arr = a.tolist()
    else:
        arr = list(a)
    if not arr:
        return []
    # Verifica che non ci siano valori negativi
    if min(arr) < 0:
        raise ValueError('Counting sort: supporta solo interi non negativi')
    # calcolo diretto del massimo
    max_value = max(arr)
    k = max_value + 1
    # limite di sicurezza per evitare OOM accidentali
    MAX_K_LIMIT = 50_000_000
    if k > MAX_K_LIMIT:
        raise MemoryError(f'k troppo grande per il counting sort: k={k} > {MAX_K_LIMIT}')
    counts = [0] * k
    for v in arr:
        counts[v] += 1
    out = []
    for val, c in enumerate(counts):
        if c:
            out.extend([val] * c)
    return out

### Teoria: counting-sort e varianti
- Complessità: tempo O(n + k), spazio O(k) addizionale oltre l'input (k = max_value - min_value + 1).
- Stabilità: la versione stabile usa conteggi cumulativi e costruisce un array di output (scorrendo l'input da destra a sinistra) per preservare l'ordine relativo dei record con la stessa chiave.
- Varianti: per payload associati usare la versione stabile; per prestazioni in Python/NumPy considerare `np.bincount` che sfrutta implementazioni C/ottimizzazioni.
- Quando evitarlo: se k è molto grande rispetto alla memoria disponibile, preferire algoritmi comparativi O(n log n).

Esempio rapido di `counting_sort` (ora senza passare il valore massimo).

In [26]:
print('counting_sort result (sample):', counting_sort(sample))

counting_sort result (sample): [4, 4, 4, 10, 21, 21, 26, 32, 34, 38, 42, 48]


## Test di correttezza
Verifica su casi semplici e edge case.

In [ ]:
def is_sorted(a):
    return all(a[i] <= a[i+1] for i in range(len(a)-1))

# più test, includendo casi con ripetizioni e range piccoli/grandi
tests = [[], [1], [2,1], [5,3,8,1,2,7], list(RNG.integers(0,50,size=20)), [0,0,0,0], list(range(10,0,-1))]
for t in tests:
    res_i = insertion_sort(t)
    res_c = counting_sort(t)
    assert is_sorted(res_i)
    assert is_sorted(res_c)
    assert sorted(list(t)) == res_i == res_c
print('Tutti i test di correttezza passati.')

### Teoria: correttezza e limitazioni pratiche
- Correttezza: entrambi gli algoritmi sono deterministici e la versione implementata di `counting_sort` produce l'output ordinato come `sorted()`.
- Limitazioni pratiche: la conversione da `numpy` a `list` (quando presente) ha costi in tempo e memoria; per dataset grandi preferire implementazioni che sfruttano operazioni `numpy` native.

## Benchmark: tempo e memoria
Misuriamo tempo medio (con deviazione) e picco di memoria Python allocata (usando tracemalloc).
Eseguiamo test su diverse combinazioni di n (dimensione) e k (massimo valore possibile) e su due distribuzioni: uniforme e clusterizzata.

In [ ]:
import tracemalloc
import math
from statistics import mean, stdev


def time_mem_function(func, args=(), repeat=5):
    times = []
    mem_peaks = []
    for _ in range(repeat):
        tracemalloc.start()
        t0 = time.perf_counter()
        try:
            func(*args)
        finally:
            t1 = time.perf_counter()
            current, peak = tracemalloc.get_traced_memory()
            tracemalloc.stop()
        times.append(t1 - t0)
        mem_peaks.append(peak)
    mean_t = float(mean(times))
    std_t = float(stdev(times)) if len(times) > 1 else 0.0
    mean_m = float(mean(mem_peaks))
    std_m = float(stdev(mem_peaks)) if len(mem_peaks) > 1 else 0.0
    return (mean_t, std_t, mean_m, std_m)

# warm-up
warm = RNG.integers(0, 100, size=200)
_ = insertion_sort(warm)
_ = counting_sort(warm)
print('Warm-up eseguito.')

# definizioni dei test (estesi)
n_values = [100, 300, 1000, 3000]
k_values = [10, 100, 1000, 10000]
repeat = 5

results = []  # lista di dicts: {dist, n, k, algo, time_mean, time_std, mem_mean, mem_std}

for k in k_values:
    for n in n_values:
        # distribuzione uniforme su [0, k]
        data_uniform = RNG.integers(0, k+1, size=n)
        # distribuzione cluster (molti valori piccoli, pochi grandi)
        small_part = RNG.integers(0, max(1, k//10)+1, size=int(n*0.9))
        big_part = RNG.integers(0, k+1, size=int(n*0.1))
        data_cluster = np.concatenate([small_part, big_part])
        for dist_name, data in [('uniform', data_uniform), ('cluster', data_cluster)]:
            # insertion sort
            mean_i, std_i, mean_mi, std_mi = time_mem_function(insertion_sort, (data,), repeat=repeat)
            results.append({'dist': dist_name, 'n': n, 'k': k, 'algo': 'insertion', 'time_mean': mean_i, 'time_std': std_i, 'mem_mean': mean_mi, 'mem_std': std_mi})
            # counting sort (proteggiamo da MemoryError)
            try:
                mean_c, std_c, mean_mc, std_mc = time_mem_function(counting_sort, (data,), repeat=repeat)
            except MemoryError as me:
                print(f'MemoryError for counting_sort with k={k}, n={n}:', me)
                mean_c = std_c = mean_mc = std_mc = math.nan
            results.append({'dist': dist_name, 'n': n, 'k': k, 'algo': 'counting', 'time_mean': mean_c, 'time_std': std_c, 'mem_mean': mean_mc, 'mem_std': std_mc})
            print(f'k={k} n={n} dist={dist_name} -> insertion {mean_i:.6f}s (±{std_i:.6f}), mem={mean_mi:.0f}B; counting {mean_c:.6f}s (±{std_c:.6f}), mem={mean_mc:.0f}B')

# grafici: per ogni k e distribuzione, tempo e memoria in subplot affiancati
for k in k_values:
    for dist in ['uniform', 'cluster']:
        subset = [r for r in results if r['k']==k and r['dist']==dist]
        ns = sorted(set(r['n'] for r in subset))
        ins = [r for r in subset if r['algo']=='insertion']
        cnt = [r for r in subset if r['algo']=='counting']
        if not ins or not cnt:
            continue
        ins_times = [next(r['time_mean'] for r in ins if r['n']==n) for n in ns]
        cnt_times = [next(r['time_mean'] for r in cnt if r['n']==n) for n in ns]
        ins_mem = [next(r['mem_mean'] for r in ins if r['n']==n) for n in ns]
        cnt_mem = [next(r['mem_mean'] for r in cnt if r['n']==n) for n in ns]

        plt.figure(figsize=(10,4))
        plt.subplot(1,2,1)
        plt.plot(ns, ins_times, label='insertion-sort', marker='o')
        plt.plot(ns, cnt_times, label=f'counting-sort (k={k})', marker='s')
        plt.xlabel('n')
        plt.ylabel('Tempo medio (s)')
        plt.title(f'Tempo - k={k} dist={dist}')
        plt.legend()
        plt.grid(True, linestyle='--', alpha=0.6)

        plt.subplot(1,2,2)
        plt.plot(ns, ins_mem, label='insertion-sort mem', marker='o')
        plt.plot(ns, cnt_mem, label='counting-sort mem', marker='s')
        plt.xlabel('n')
        plt.ylabel('Memoria picco (bytes)')
        plt.title(f'Memoria (picco) - k={k} dist={dist}')
        plt.legend()
        plt.grid(True, linestyle='--', alpha=0.6)

        fname = f'compare_insertion_counting_k{k}_dist_{dist}.png'
        plt.tight_layout()
        plt.savefig(fname)
        print('Salvato', fname)
        plt.show()

## Teoria: benchmarking e misurazioni
- tracemalloc misura le allocazioni del runtime Python (heap) e il picco di memoria allocata dalle strutture Python durante l'esecuzione della funzione; NON misura il consumo di memoria a livello di processo (RSS). Per misurare l'RSS usare `psutil.Process().memory_info().rss` o leggere `/proc/<pid>/status` su Linux.
- Per confronti affidabili: ripetere ogni misura più volte, scartare warm-up della JIT (se presente) e usare la mediana o la media con deviazione standard per riassumere i risultati.
- Attenzione all'overhead delle misure: tracemalloc e strumenti di profiling introducono un piccolo overhead; confronti puramente temporali dovrebbero essere fatti sia con che senza misurazione della memoria per valutare l'impatto.

## Raccomandazioni pratiche
- Proteggere la versione di counting sort con una soglia di sicurezza su k (per evitare OOM accidentali). Qui usiamo `MAX_K_LIMIT` come valore prudente; puoi adattarlo in base alla RAM disponibile.
- Per dataset grandi usare alternative numpy-native (`np.bincount`) o algoritmi O(n log n) se k è grande.
- Se servono dati stabili con payload, usare la versione stabile di counting sort o alternative come radix sort.

## Conclusione rapida
- insertion-sort cresce ~O(n^2).
- counting-sort è O(n + k).
